In [197]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import math
from statistics import mode
import tensorflow as tf

In [198]:
# Read the CSV file to classify whether the student has a A score 
df_matA = pd.read_csv('mat_A.csv', sep=',')
data = df_matA.to_numpy()[:, 1:]

# standarize the data 
indices_to_standarize = []
for i, feature in enumerate(data[0]):
    if type(feature) != str and i != (data.shape[1]-1):
        indices_to_standarize.append(i)

for i in indices_to_standarize:
    m = np.mean(data[:, i])
    std = np.std(data[:, i])
    data[:, i] = (data[:, i]-m)/std

# split in to training and testing
X_train, X_test, y_train, y_test = train_test_split(data[:, :-1], data[:, -1], test_size=0.1, random_state=0)

In [199]:
def dist(x1, x2):
    total_dist = 0
    
    for i, feature in enumerate(x1):
        # for numerical values, the distance is manhattan
        if type(feature) != str:
            total_dist += abs(x1[i] - x2[i])
        # for categorical values, the distance is hamming
        else:
            if x1[i] != x2[i]:
                total_dist += 1
    return total_dist

In [200]:
def find_min(dists, k):
    x_sorted = np.sort(dists)
    min_values = x_sorted[:k]
    indices = [dists.index(value) for value in min_values]
    return indices
    

In [201]:
# knn
def knn(X_train, X_test, y_train, y_test, k):

    y_pred = []

    # for each test data
    for x in X_test:
        
        # get an array represent the distance between x and each training data
        distances = []
        for x_train in X_train:
            distances.append(dist(x, x_train))
        
        # find the k lowest distance
        lowest_indices = find_min(distances, k)
        
        # majority vote from these k
        pred = mode(y_train[lowest_indices])
        
        # append the predicted result to the prediction array
        y_pred.append(pred)
        
    accuracy = sum(y_pred == y_test)/len(y_pred)
    return accuracy

In [202]:
knn(X_train, X_test, y_train, y_test, 7)

0.9

In [203]:
type('yes')

str

In [204]:
# ANN
# process data by change yes/no to 0/1 and delete all categorical data:
new_data = data.copy()
columns_to_delete = []
for sample in new_data:
    for i in range(len(sample)):
        if sample[i] == 'yes':
            sample[i] = 1
        elif sample[i] == 'no':
            sample[i] = 0
for i in range(len(new_data[0])):
    if type(new_data[i][i]) == str:
        columns_to_delete.append(i)

new_data = np.delete(new_data, columns_to_delete, axis = 1)
# split in to training and testing
X_train, X_test, y_train, y_test = train_test_split(new_data[:, :-1], new_data[:, -1], test_size=0.1, random_state=0)

In [205]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [206]:


# Initialize the ANN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=64, activation='relu', input_shape=(23,)))
model.add(tf.keras.layers.Dense(units=32, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - accuracy: 0.7084 - loss: 0.5926
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - accuracy: 0.8166 - loss: 0.4713
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - accuracy: 0.8508 - loss: 0.3915
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - accuracy: 0.8923 - loss: 0.3270
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 678us/step - accuracy: 0.9010 - loss: 0.2826
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.9151 - loss: 0.2545
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - accuracy: 0.9344 - loss: 0.2077
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - accuracy: 0.9098 - loss: 0.2162
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step - accuracy: 0.9328 - loss: 0.1803
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.9437 - loss: 0.1621
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.9088 - loss: 0.1666
Epoch 12/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0